<body>
    <div>
        <h2 style='font-family: cursive; font-size: 40px;'>Factorizacion Cholesky</h2>
<!--         <img src= alt=middle style="height: 340px"> -->
    </div>
</body>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Formulación matemática</h3>

$A=L*L^T$

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Valores iniciales</h3>

A: Matriz de tamaño mxm que sea SDP.
<br>
b: Vector de tamaño m.

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Ventajas y desventajas</h3>

<table style="width:100%; text-align: left; font-size: 14px">
    <tr style="text-align: left;">
        <th style="text-align: left;">Ventajas</th>
        <th style="text-align: left;">Desventajas</th>
    </tr>
    <tr>
        <td style="text-align: left;">Es un metodo que utiliza pocos recursos del computador haciendo más rapido el proceso de datos.</td>       
        <td style="text-align: left;">Solo se puede aplicar cuando la matriz es SDP.</td>
    </tr>
</table>

<h3 style="font-family: 'Courier New'; font-size:25px"
    >Pseudocódigo</h3>

<ol>
    <li>Verificar que A es SDP</li>
    <li>Para j=1:m</li>
    <li>&emsp;$l_{jj} = \sqrt{a_{jj} -\sum\limits_{k=1}^{j-1} l_{jk}^2}$</li>
    <li>&emsp;Para i=j:m</li>
    <li>&emsp;&emsp;$l_{ij} = \sqrt{a_{ij} -\sum\limits_{k=1}^{j-1} l_{jk}*l_{ik}}$</li>
    <li>&emsp;Fin Para</li>
    <li>Fin Para</li>
    <li>Y = hacer sustituciona hacia delante de $l$ y b</li>
    <li>x = hacer sustituciona hacia atras de $l^T$ y Y</li>
    <li>Retornar x</li>
</ol>

<h3 style="font-family: 'Courier New'; font-size:25px">Octave</h3>

In [20]:
function [error] = checkSDP(a, m, n)
%    Verifica que la matriz sea simetrica
%    :param a: una matriz
%    :param m: tamaño de la columna
%    :param n: tamaño de la columna
%    :return: verdadero si lo es o falso si no
    error = 0;
    %verifica que sea cuadrada
    if  m != n
        error = 1;
        disp('Error: La matriz no es cuadrada.');
    end
    %Verifica si no es simetrica
    if !issymmetric(a)
        error = 1;
        disp('Error: La matriz no es simetrica.');
    else
        for i=1:m
            %Verifica que sea definida positiva
            if det(a(1:i, 1:i)) < 0
                error = 1;
                disp('Error: La matriz no es definida positiva.');
            end
        end
    end
end

function [l] = cholesky_aux(a, m, n)
%    Calcula una matriz inferior con el metodo de cholesky
%    :param a: una matriz
%    :param m: tamaño de la columna
%    :param n: tamaño de la columna
%    :return: Una matriz triangular inferior
    l(m,m) = 0;
    l(1,1) = sqrt(a(1, 1));

    for i=2:m
        l(i,1) = a(i, 1)/l(1,1);

        for j=2:i-1
            l_sum = 0;
            
            for k=1:j
                l_sum = l_sum + l(j,k) * l(i,k);
            end
            l(i,j) = (a(i,j) - l_sum)/l(j,j);
        end
        l_pow = 0;
        for k=1:i
            l_pow = l_pow + l(i,k)**2;
        end
        l(i,i) = sqrt(a(i,i) - l_pow);
    end
end

function [x] = backwardSubstitution(a, m, n)
%    Funcion para obtener la sustitucion hacia atras de una matriz superior
%    :param a: una matriz superior
%    :param m: tamaño de las filas de la matriz
%    :param n: tamaño de las columnas de la matriz
%    :return: matriz con el valor de cada x
    %Calcula el valor de x_m
    x_i(m) = [a(m,n)/a(m,m)];
    for i=m:-1:1
        x_sum = 0;
        %Suma la multiplicacion de la contante junto con la variable encontrada
        for j=i+1:m
            x_sum = x_sum + a(i, j) * x_i(j);
        end
        %Obtiene el valor de x_i
        x_i(i) = (a(i, n)-x_sum)/a(i,i);
    end
    x = x_i';
end

function [y] = forwardSubstitution(a, m, n)
%     Funcion para obtener la sustitucion hacia delante de una matriz inferior
%     :param a: una matriz inferior
%     :param m: tamaño de las filas de la matriz
%     :param n: tamaño de las columnas de la matriz
%    :return: matriz con el valor de cada y
    x_i(m) = 0;
    %Calcula el valor de x_0
    x_i(1) = a(1,n)/a(1,1);
    for i=2:m
        x_sum = 0;
        %Suma la multiplicacion de la contante junto con la variable encontrada
        for j=1:i
            x_sum = x_sum + a(i, j) * x_i(j);
        end
        %Obtiene el valor de x_i
        if a(i,i) != 0
            x_i(i) = (a(i, n)-x_sum)/a(i,i);
        else
            x_i(i) = 0;
        end
    end
    y = x_i';
end

function [x] = cholesky(a, b)
%     Metodo Factorizacion de Cholesky para resolver un sistema A x = B
%     :param a: una matriz
%     :param b: un vector
%     :return: matriz x que resuelve el sistema A x = b
    %Obtiene el tamaño de la matriz
    [m,n] = size(a);
    %Verifica que cumpla que sea SDP
    if checkSDP(a, m, n)
        x = 0;
    else
        %Calcula la matriz inferior con el metodo de cholesky
        l = cholesky_aux(a, m, n);
        l_2 = [l b];
        %Obtiene la matriz de soluciones
        [y] = forwardSubstitution(l_2, m, n + 1);
        l_t = [l' y];
        [x] = backwardSubstitution(l_t, m, n + 1);
    end    
end

<h3 style="font-family: 'Courier New'; font-size:25px">Python</h3>

In [17]:
from sympy import Matrix
from math import sqrt

def transpose(a, n): 
    """
    Obtiene la transpuesta de una matriz
    :param a: una matriz
    :param n: tamaño de la columna
    :return: matriz transpuesta
    """
    tr = []
    for i in range(n):
        tr.append([0] * n)
    for i in range(n): 
        for j in range(n): 
            #Invierte los valores de la matriz en tr
            tr[i][j] = a[j][i]
    return tr

def issymmetric(a, n):
    """
    Verifica que la matriz sea simetrica
    :param a: una matriz
    :param n: tamaño de la columna
    :return: verdadero si lo es o falso si no
    """
    for i in range(n): 
        for j in range(n): 
            if (a[i][j] != a[j][i]): 
                return False
    return True


def checkSDP(a, m, n):
    """
    Verifica que la matriz sea simetrica
    :param a: una matriz
    :param m: tamaño de la columna
    :param n: tamaño de la columna
    :return: verdadero si lo es o falso si no
    """
    error = False
    #verifica que sea cuadrada
    if m != n:
        error = True
        print('Error: La matriz no es cuadrada.')
    #Verifica si no es simetrica
    if not(issymmetric(a, n)):
        error = True
        print('Error: La matriz no es simetrica.')
    else:
        a = Matrix(a)
        #Verifica que sea definida positiva
        if not(a.is_positive_definite):
            error = 1
            print('Error: La matriz no es definida positiva.')
    return error


def cholesky_aux(a, m, n):
    """
    Calcula una matriz inferior con el metodo de cholesky
    :param a: una matriz
    :param m: tamaño de la columna
    :param n: tamaño de la columna
    :return: Una matriz triangular inferior
    """
    l = []
    for i in range(m):
        l.append([0] * m)
    l[0][0] = sqrt(a[0][0])
    
    for i in range(1, m):
        l[i][0] = a[i][0] / l[0][0]
        for j in range(1, i):
            l_sum = 0
            for k in range(j):
                l_sum += l[j][k] * l[i][k]
            l[i][j] = (a[i][j] - l_sum) / l[j][j]
        l_pow = 0
        for k in range(i):
            l_pow += l[i][k]**2
        l[i][i] = sqrt(a[i][i] - l_pow)
    return l
    
    
def backwardSubstitution(a, m, n):
    """
    Funcion para obtener la sustitucion hacia atras de una matriz superior
    :param a: una matriz superior
    :param m: tamaño de las filas de la matriz
    :param n: tamaño de las columnas de la matriz
    :return: matriz con el valor de cada x
    """
    x_i = [0] * m
    #Calcula el valor de x_m
    x_i[m-1] = a[m-1][n-1]/a[m-1][m-1]
    for i in range(m-1, -1, -1):
        x_sum = 0
        #Suma la multiplicacion de la contante junto con la variable encontrada
        for j in range(i+1, m):
            x_sum += a[i][j] * x_i[j]
        #Obtiene el valor de x_i
        x_i[i] = (a[i][m] - x_sum)/a[i][i]
    return x_i
    

def forwardSubstitution(a, m, n):
    """
    Funcion para obtener la sustitucion hacia delante de una matriz inferior
    :param a: una matriz inferior
    :param m: tamaño de las filas de la matriz
    :param n: tamaño de las columnas de la matriz
    :return: matriz con el valor de cada y
    """
    x_i = [0] * m
    #Calcula el valor de x_0
    x_i[0] = a[0][n-1]/a[0][0]
    for i in range(1, m):
        x_sum = 0
        #Suma la multiplicacion de la contante junto con la variable encontrada
        for j in range(0, i):
            x_sum += a[i][j] * x_i[j]
        #Obtiene el valor de x_i
        if a[i][i] != 0:
            x_i[i] = (a[i][m] - x_sum)/a[i][i]
        else:
            x_i[i] = 0
    return x_i


def appendColumn(a, b):
    """
    Adjunta una columna B a una matriz A
    :param a: una matriz
    :return: matriz con la columna B adjunta
    """
    m = len(a)
    for i in range(m):
        a[i].append(b[i])
    return a


def cholesky(a, b):
    """
    Metodo Factorizacion de Cholesky para resolver un sistema A x = B
    :param a: una matriz
    :param b: un vector
    :return: matriz x que resuelve el sistema A x = b
    """
    #Obtiene el tamaño de la matriz
    m = len(a) 
    n = len(a[0])
    #Verifica que cumpla que sea SDP
    if checkSDP(a, m, n):
        x = 0
    else:
        #Calcula la matriz inferior con el metodo de cholesky
        l = cholesky_aux(a, m, n)
        #Obtiene la matriz de soluciones
        y = forwardSubstitution(appendColumn(l, b), m, n + 1)
        x = backwardSubstitution(appendColumn(transpose(l, m), y), m, n + 1)
    return x

<h3 style="font-family: 'Courier New'; font-size:25px">Ejemplo Numérico</h3>

<h5 style="font-family: 'Courier New'; font-size:16px">Python</h5>

In [22]:
a = [[25, 15, -5, -10], [15, 10, 1, -7], [-5, 1, 21, 4], [-10, -7, 4, 18]]
b = [-25, -19, -21, -5]
print(cholesky(a, b))

[-1.0, -1.0, -1.0, -1.0]


<h5 style="font-family: 'Courier New'; font-size:16px">Octave</h5>

In [21]:
a = [25 15 -5 -10; 15 10 1 -7; -5 1 21 4; -10 -7 4 18];
b = [-25; -19; -21; -5];

x = cholesky(a, b)

x =

  -1
  -1
  -1
  -1

